In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
import networkx as nx

Загружаем данные по тройкам. Данные хранятся в формате .xlsx, для работы с ними используем модуль pandas. **Pandas** (Python Data Analysis Library) — библиотека языка Python для [удобных] обработки и анализа данных. Загрузка данных в **pandas** происходит в специальный объект типа **DataFrame**:

In [ ]:
dt=pd.read_excel('eng_rat.xlsx') # for rat test with words in G
print(dt)

Как видно, данные представляют из себя таблицу, где строка — объект, столбец — признак. Для экономии места можно выводить заданное количество первых строк объекта при помощи метода head():

In [ ]:
dt.head(10)

По данным можно индексироваться при помощи номеров строк/столбцов или названий признаков:

In [ ]:
dt[2:5]

In [ ]:
dt.iloc[1:5, 0:4]

In [ ]:
dt[['Word1', 'Word2', 'Word3','Answer','15s_acc']].head()

Постмотрим на распределение числа верно ответивших на тест за 15 секунд:

In [ ]:
dt['15s_acc'].hist(bins = 30)
plt.xlabel(' accuracy 15 sec', fontsize=14)

И распределение времен ответа:

In [ ]:
dt['15s_st'].hist(bins = 30)
plt.xlabel(' accuracy 15 sec', fontsize=14)

Есть ли корреляция между точностью решения задачи и временем?

In [ ]:
plt.scatter(dt['15s_acc'],dt['15s_st'])
plt.xlabel(' accuracy 15 sec', fontsize=14)
plt.ylabel(' answer time', fontsize=14)

Определим сложность теста по проценту решивших тест верно:

In [ ]:
y=[]
for row in dt.itertuples(): # Iterate over DataFrame
    acc=row._9
    if(acc<=32.0): y.append(0)
    else: 
        if(acc<=64.0): y.append(1)
        else: y.append(2)
dt['Comp']=y 

dt.head()

Теперь можно посмотреть статистику тестов внутри одного класса сложности: 

In [ ]:
acc,err_acc=[],[]
mst,err_st=[],[]
for c in [0,1,2]:
    t=dt[(dt['Comp']==c)]['15s_acc'].mean()
    st=dt[(dt['Comp']==c)]['15s_acc'].std()
    acc.append(t)
    err_acc.append(st)
    t=dt[(dt['Comp']==c)]['15s_st'].mean()
    st=dt[(dt['Comp']==c)]['15s_st'].std()
    mst.append(t)
    err_st.append(st)

In [ ]:
fig, ax = plt.subplots(nrows=1, sharey=False,ncols=2, figsize=(12, 5))
x_pos = np.arange(3)

ax[0].bar(x_pos, acc, yerr=err_acc, align='center', alpha=0.5, ecolor='black', capsize=10)
ax[0].set_xlabel('complexity', fontsize=14)
ax[0].set_ylabel('accuracy', fontsize=14)

ax[1].bar(x_pos, mst, yerr=err_st, align='center', alpha=0.5, ecolor='black', capsize=10)
ax[1].set_xlabel('complexity', fontsize=14)
ax[1].set_ylabel('answer time', fontsize=14)

# Топологические свойства RAT в сети

Загружаем сеть через dataframe и выводим основные характеристики сети:

In [1]:
net=pd.read_excel('assoc_eng2.xlsx')
G0=nx.from_pandas_dataframe(net,'source', 'target', edge_attr=True, create_using=nx.DiGraph())
print('Nodes:',G0.number_of_nodes())
print('Edges:',G0.number_of_edges())
print('Density:',nx.density(G0))
print('Transitivity', nx.transitivity(G0))

Мы в первую очередь будем интересоваться путями между словами в RAT, поэтому начнем с проверки на связность:

In [ ]:
print(nx.is_strongly_connected(G0))
print(len(max(nx.strongly_connected_components(G0), key=len)))

В отличие от сети ca-netscience сеть ассоциаций направленна, и для существования путей и алгоритмов распространения активации такое свойство сети оказывается ключевым. Для сравнения посмотрим на ненаправленную сеть, полученнуюи из направленной:

In [ ]:
H=G0.to_undirected()
print('Nodes:',H.number_of_nodes())
print('Edges:',H.number_of_edges())
print('Density:',nx.density(H))
print('Transitivity', nx.transitivity(H))

Почему ребер в сети меньше, чем в направленной?)))

In [ ]:
print(nx.is_connected(H))
print(len(max(nx.connected_components(H), key=len)))

# Задание

1) Исследуйте топологические свойства сети ассоциаций. Помните, что сеть направленная и ряд свойств отличается от анологичных, определенных на ненаправленной сети. 

2) Помимо того что сеть ассоциаций направленна, она еще является взвешенной,т.е. каждое ребро имеет вес. Постройте распределение значений веса ребер. Проанализируйте как изменится топология сети, если убирать ребра малого веса. Для этого можно ввести порог $w_0$ и удалять все связи с весом $w_{i,j}<w_0$. Постройте зависимости топологических характеристик от значения $w_0$. 

3) Исследуйте топологические свойства вершин, входящих в RAT, на сети. Есть ли корреляция между сложностью RAT/временем решения RAT и топологическими свойствами вершин? Придумайте, какие топологические характеристики могут отражать сложность теста. Как можно учесть направленность и взвешенность сети ассоциаций? 

4) Оформите результаты анализа в виде небольшой презентации (3-4 слайда).

